## Parameter Sweep Analysis: CSL License Dynamics

In this section, we systematically explore **how key parameters of the CSL license affect open-source ecosystem outcomes**. Specifically, we vary:

1. **Contributor Share Percentage (`contributor_share_pct`)**  
   - The fraction of revenue from organizations that is directed back to a shared contributor fund.  
   - Higher values mean corporations contribute more back to the community.

2. **Barrier to Entry (`barrier_to_entry`)**  
   - A restriction that reduces the probability that large organizations can adopt the software.  
   - Simulates the effect of CSL’s “corporate control” mechanisms or stricter compliance requirements.  

3. **Fork Penalty (`fork_penalty`)**  
   - Adjusted automatically based on barrier level, representing the difficulty or cost for large organizations to fork and bypass contributions.  

---

### What the Sweep Does

We run simulations for **all combinations** of `contributor_share_pct` and `barrier_to_entry`. For each scenario, we track:

- **Final Contributor Fund**  
  Total funds accumulated by the community after the simulation period.  
- **Mean Active Developers**  
  Average number of active developers contributing over time.  
- **Final Adoption**  
  Total adoption by all agents (Individuals, Small Orgs, Large Orgs) at the end of the simulation.  
- **Final Market Concentration**  
  Measures how concentrated revenue is among adopters (a proxy for monopolization risk).  

By sweeping through the parameter space, we can **see which combinations best support a sustainable open-source ecosystem**.

---

### Reading the Heatmap

The heatmap shows the **final contributor fund** for each combination of parameters:

- **Y-axis (`Contributor Share %`)**: How much corporations contribute to the community.  
- **X-axis (`Barrier to Entry`)**: How much adoption by large corporations is restricted.  
- **Color intensity**: Higher values indicate **larger funds accumulated**, signaling stronger community support.

**Intuition for Engineers**:

- **Top-right corner** (high share, high barrier): Large contributions but slow adoption; fund may grow rapidly, but fewer organizations can adopt.  
- **Bottom-left corner** (low share, low barrier): Quick adoption but weak community support; contributor fund remains low.  
- **Balanced middle areas**: Offer a compromise between adoption and community sustainability.

---

This sweep allows us to **quantify trade-offs** and make informed decisions about:

- How aggressively to require corporate contributions.  
- How strict license barriers should be to maximize fund growth **without excessively limiting adoption**.  
- How license parameters influence developer engagement and ecosystem health.


In [ ]:
# 04_parameter_sweeps.ipynb
import pandas as pd
from itertools import product
# Use simulate() from previous notebook (import or re-define)

share_vals = [0.01,0.05,0.10,0.15,0.20]
barrier_vals = [0.0,0.1,0.3,0.5,0.6]
sweep_results = []

for share, barrier in product(share_vals, barrier_vals):
    p = base_params['CSL'].copy()
    p['contributor_share_pct'] = share
    p['barrier_to_entry'] = barrier
    p['fork_penalty'] = min(0.9, 0.3 + barrier*0.8)
    df = simulate(p)
    sweep_results.append({
        'share':share,
        'barrier':barrier,
        'final_fund':df['contributor_fund'].iloc[-1],
        'mean_active_devs':df['active_developers'].mean(),
        'final_adoption':df['total_adoption'].iloc[-1],
        'final_market_conc':df['market_concentration'].iloc[-1]
    })

sweep_df = pd.DataFrame(sweep_results)
sweep_df.to_csv('csl_parameter_sweep.csv',index=False)

# Heatmap visualization
import matplotlib.pyplot as plt
pivot = sweep_df.pivot(index='share', columns='barrier', values='final_fund')
fig, ax = plt.subplots()
c = ax.imshow(pivot.values, aspect='auto', origin='lower')
ax.set_xticks(range(len(pivot.columns))); ax.set_xticklabels(pivot.columns)
ax.set_yticks(range(len(pivot.index))); ax.set_yticklabels([f"{s:.0%}" for s in pivot.index])
ax.set_xlabel('Barrier to Entry'); ax.set_ylabel('Contributor Share %')
ax.set_title('Final Contributor Fund after T steps')
fig.colorbar(c, ax=ax)
plt.show()
